<a href="https://colab.research.google.com/github/SomdeepAcharyya/Fake-News-classifiaction/blob/main/Trust%26Believe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import files
import io
import pandas as pd
import spacy
from textblob import TextBlob
from spacy.lang.en import English
nlp = English()
import csv
from itertools import chain
from spacy.lang.en.stop_words import STOP_WORDS
import nltk
nltk.download('wordnet')
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nlp = spacy.load('en')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
spc = spacy.load('en_core_web_sm')
import http.client
import urllib.parse
from urllib.request import urlopen
import math
!pip install tweet-preprocessor
import preprocessor as p
p.set_options(p.OPT.EMOJI, p.OPT.SMILEY, p.OPT.MENTION, p.OPT.HASHTAG, p.OPT.ESCAPE_CHAR, p.OPT.RESERVED)
import numpy as np

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
uploaded1 = files.upload()

Saving boston_bombings.csv to boston_bombings (1).csv


In [ ]:
df_tweets = pd.read_csv('aug_complete.csv')
df_user = pd.read_csv('boston_bombings.csv')
df_tweets = df_tweets.dropna()
df_tweets = df_tweets.reset_index(drop=True)
df_user = df_user.dropna()
df_user = df_user.reset_index(drop=True)

In [ ]:
path = r'/content/drive/MyDrive/Colab Notebooks/processed.csv'
with open(path, encoding="utf-8", errors='ignore') as infile:
  df = pd.read_csv(infile)

In [ ]:
df_tweets = df

In [ ]:
df_tweets.urls

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
      ... 
995    NaN
996    NaN
997    NaN
998    NaN
999    NaN
Name: urls, Length: 1000, dtype: object

In [ ]:
lemmatizer = WordNetLemmatizer()
punctuation = re.compile(r'[-.?&!,:;()|0-9]\/`'"'")
def get_lemma_words(t):
    t = t.lower()
    a = nlp(t)
    lemmat= " "
    stop_words = set(stopwords.words('english'))  
    word_tokens = word_tokenize(p.clean(t)) 
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    punc_words = []
    for words in filtered_sentence:
        word =  punctuation.sub("", words)
        if len(word) > 0:
          punc_words.append(word)
    for word in punc_words:
        lemmat = lemmat + " " + (lemmatizer.lemmatize(word))
    #print('y')
    return lemmat

In [ ]:
#processing text by preprocessing & lemmatization

processed_text = []
for i in range(len(df_tweets)):
  x = df_tweets['text'][i]
  y = get_lemma_words(x)
  processed_text.append(y)
df_tweets['processed_text'] = processed_text

In [ ]:
df_tweets.columns

Index(['coordinates', 'created_at', 'hashtags', 'media', 'urls',
       'favorite_count', 'id', 'in_reply_to_screen_name',
       'in_reply_to_status_id', 'in_reply_to_user_id', 'lang', 'place',
       'possibly_sensitive', 'retweet_count', 'retweet_id',
       'retweet_screen_name', 'source', 'text', 'tweet_url', 'user_created_at',
       'screen_name', 'user_default_profile_image', 'description',
       'favourites_count', 'followers_count', 'friends_count', 'listed_count',
       'location', 'user_name', 'user_screen_name', 'statuses_count',
       'time_zone', 'user_urls', 'verified', 'processed_text', 'domain',
       'intersection_words', 'sentiment', 'archived_tweets',
       'followers_friends_ratio'],
      dtype='object')

In [ ]:
# no of tweets

df_tweets['archived_tweets'] = 0

for i in range(len(df_tweets)):
  x = df_tweets['screen_name'][i]
  for j in range(len(df_tweets)):
        if x == df_tweets['screen_name'][j]:
          df_tweets['archived_tweets'][j] = df_tweets['archived_tweets'][j] + 1

df_tweets['archived_tweets']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

0      1
1      1
2      1
3      1
4      1
      ..
995    1
996    1
997    1
998    1
999    1
Name: archived_tweets, Length: 1000, dtype: int64

In [ ]:
# followers-friends ratio

followers_friends = []
for i in range(len(df_tweets)):
  x = df_tweets['followers_count'][i]
  y = df_tweets['friends_count'][i]
  followers_friends.append(x/y)
df_tweets['followers_friends_ratio'] = followers_friends
df_tweets['followers_friends_ratio']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in long_scalars
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0      0.500000
1      1.121908
2      2.181818
3      0.307071
4      0.995540
         ...   
995    0.612069
996    2.238956
997    0.693069
998         inf
999    0.250109
Name: followers_friends_ratio, Length: 1000, dtype: float64

In [ ]:
# no of retweets

df_tweets['retweets'] = 0
df_tweets['retweets_norm'] = 0.000

for i in range(len(df_tweets)):
  x = df_tweets['screen_name'][i]
  z = df_tweets['text'][i]
  if '@' in z:
    for j in range(len(df_tweets)):
        if x == df_tweets['screen_name'][j]:
          df_tweets['retweets'][j] = df_tweets['retweets'][j] + 1

for j in range(len(df_tweets)):
   df_tweets['retweets_norm'][j] = df_tweets['retweets'][j]/df_tweets['archived_tweets'][j]

df_tweets['retweets']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats i

0      1
1      0
2      0
3      1
4      1
      ..
995    1
996    0
997    1
998    1
999    0
Name: retweets, Length: 1000, dtype: int64

In [ ]:
# no of likes

df_tweets['likes'] = 0
df_tweets['likes_norm'] = 0.00

for i in range(len(df_tweets)):
  x = df_tweets['screen_name'][i]
  for j in range(len(df_tweets)):
        if x == df_tweets['screen_name'][j]:
          df_tweets['likes'][j] = df_tweets['likes'][j] + df_tweets['favorite_count'][i]

for j in range(len(df_tweets)):
   df_tweets['likes_norm'][j] = df_tweets['likes'][j]/df_tweets['archived_tweets'][j]

df_tweets['likes']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats i

0      0
1      0
2      0
3      0
4      0
      ..
995    0
996    0
997    0
998    0
999    0
Name: likes, Length: 1000, dtype: int64

In [ ]:
df_tweets['favorite_count']

0      0
1      0
2      0
3      0
4      0
      ..
995    0
996    0
997    0
998    0
999    0
Name: favorite_count, Length: 1000, dtype: int64

In [ ]:
# no of urls

df_tweets['url_count'] = 0.000

for i in range(len(df_tweets)):
  x = df_tweets['screen_name'][i]
  for j in range(len(df_tweets)):
        if x == df_tweets['screen_name'][j]:
          df_tweets['url_count'][j] = df_tweets['url_count'][j] + 0 if (df_tweets['urls'][i] == np.nan) else df_tweets['urls'][i]

for j in range(len(df_tweets)):
   df_tweets['urls'][j] = df_tweets['urls'][j]/df_tweets['archived_tweets'][j]

df_tweets['urls']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

TypeError: ignored

In [ ]:
type(df_tweets['urls'][99])

float

In [ ]:
# no of original

df_tweets['original_post'] = 0
df_tweets['original_post_norm'] = 0.000

for j in range(len(df_tweets)):
          df_tweets['original_post'][j] = (df_tweets['archived_tweets'][j] - df_tweets['retweets'][j])
          df_tweets['original_post_norm'][j] = df_tweets['original_post'][j] / df_tweets['archived_tweets'][j]

df_tweets['original_post_norm']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in

0      0.0
1      1.0
2      1.0
3      0.0
4      0.0
      ... 
995    0.0
996    1.0
997    0.0
998    0.0
999    1.0
Name: original_post_norm, Length: 1000, dtype: float64

In [ ]:
# no of social_reputation

df_tweets['social_reputation'] = 0.000

for j in range(len(df_tweets)):
          df_tweets['social_reputation'][j] = (math.log(1+df_tweets['followers_count'][j])*(1+df_tweets['followers_count'][j]))+math.log(1+df_tweets['statuses_count'][j])-math.log(1+df_tweets['followers_count'][j])

df_tweets['social_reputation']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/inde

0      2498.511573
1      4108.729179
2       317.015969
3       771.255927
4      7841.755311
          ...     
995    1612.336900
996    7834.456223
997    1134.095539
998    6321.502708
999    3654.745340
Name: social_reputation, Length: 1000, dtype: float64

In [ ]:
# calculate h_index

df_tweets['h_index'] = 0.00
df_tweets['rn_ln'] = 0.000
tweets = []
for i in range(len(df_tweets)):
  rows = []
  x = df_tweets['screen_name'][i]
  for j in range(len(df_tweets)):
    if x == df_tweets['screen_name'][j]:
      rows.append(df_tweets['processed_text'])
      df_tweets['rn_ln'][i] = df_tweets['retweets'][i]/(df_tweets['likes'][j] if not (df_tweets['likes'][j] == 0) else 1)

df_tweets['rn_ln']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats i

0      1.0
1      0.0
2      0.0
3      1.0
4      1.0
      ... 
995    1.0
996    0.0
997    1.0
998    1.0
999    0.0
Name: rn_ln, Length: 1000, dtype: float64

In [ ]:
# calculate sentiment

def get_polarity(a):
  analysis = TextBlob(a).polarity
  return analysis
def get_subjectivity(a):
  analysis = TextBlob(a).subjectivity
  return analysis

df_tweets['p_t'] = 0
df_tweets['n_t'] = 0
df_tweets['b_t'] = 0

for i in range(len(df_tweets)):
  words = []
  p_words = 0
  n_words = 0
  x = df_tweets['screen_name'][i]
  y = df_tweets['processed_text'][i]
  z = (get_polarity(y)+get_subjectivity(y))/2
  for k in range(len(df_tweets)):
    if x == df_tweets['screen_name'][k]:
      if z > 0:
        df_tweets['p_t'][k] = df_tweets['p_t'][k]+1
      elif z < 0:
        df_tweets['n_t'][k] = df_tweets['n_t'][k]+1 
      else:
        df_tweets['b_t'][k] = df_tweets['b_t'][k]+1     

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
df_tweets.to_csv( "/content/drive/MyDrive/Colab Notebooks/continue2.csv", index=False)

In [ ]:
# calculate sentiment

df_tweets['sentiment_score'] = 0.000

for i in range(len(df_tweets)):
  df_tweets['sentiment_score_tnb'][i]= (df_tweets['p_t'][i]+df_tweets['b_t'][i])/(df_tweets['p_t'][i]+df_tweets['b_t'][i]+df_tweets['n_t'][i])

df_tweets['sentiment_score_tnb']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/inde

0      1.0
1      1.0
2      1.0
3      1.0
4      1.0
      ... 
995    1.0
996    1.0
997    1.0
998    1.0
999    1.0
Name: sentiment_score, Length: 1000, dtype: float64

In [ ]:
# calculate tweet credibility

df_tweets['tweet_cred'] = 0.000

for i in range(len(df_tweets)):
  df_tweets['tweet_cred'][i]= (df_tweets['retweets_norm'][i]+df_tweets['sentiment_score'][i])/2*df_tweets['original_post_norm'][i]
df_tweets['tweet_cred']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/inde

0      0.0
1      0.5
2      0.5
3      0.0
4      0.0
      ... 
995    0.0
996    0.5
997    0.0
998    0.0
999    0.5
Name: tweet_cred, Length: 1000, dtype: float64

In [ ]:
# calculate influence score

df_tweets['influence_score'] = 0.000

for i in range(len(df_tweets)):
  df_tweets['influence_score'][i]= (df_tweets['tweet_cred'][i]+df_tweets['likes_norm'][i]+df_tweets['rn_ln'][i]+df_tweets['social_reputation'][i])/4
df_tweets['influence_score']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/inde

0       624.877893
1      1027.307295
2        79.378992
3       193.063982
4      1960.688828
          ...     
995     403.334225
996    1958.739056
997     283.773885
998    1580.625677
999     913.811335
Name: influence_score, Length: 1000, dtype: float64

In [ ]:
df_tweets['influence_score']

0       624.877893
1      1027.307295
2        79.378992
3       193.063982
4      1960.688828
          ...     
995     403.334225
996    1958.739056
997     283.773885
998    1580.625677
999     913.811335
Name: influence_score, Length: 1000, dtype: float64

In [ ]:
tnb = df_tweets[['screen_name', 'influence_score']]
tnb.to_csv( "/content/drive/MyDrive/Colab Notebooks/tnb.csv", index=False, encoding='utf-8-sig')

In [ ]:
tnb = df_tweets
tnb.to_csv( "/content/drive/MyDrive/Colab Notebooks/boston_1k.csv", index=False, encoding='utf-8-sig')

In [ ]:
class my_dictionary(dict):
  
    # __init__ function
    def __init__(self):
        self = dict()
          
    # Function to add key:value
    def add(self, key, value):
        self[key] = value
        

In [ ]:
tnb = my_dictionary()
for i in range(len(df_user)):
  tnb.add(df_user['screen_name'][i], df_user['influence_score'][i])

In [ ]:
tnb_df = df_user[['screen_name', 'influence_score']]

In [ ]:
tnb_df

In [ ]:
tnb_df.to_csv('tnb.csv', sep=',', index = False)
files.download('tnb.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>